# Prep for meeting using Exa.ai search https://exa.ai/

In [1]:
!pip -q install crewai duckduckgo-search
!pip -q install  decouple langchain-exa exa_py==1.0.7 langchain-openai

# install crewai[tools] with "pip install 'crewai[tools]'"

# crewai==0.22.5


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!py -3.11 -m pip install --upgrade pip

In [3]:
!pip show crewai

Name: crewai
Version: 0.22.5
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: D:\repos\ai-agents-crewai\.venv\Lib\site-packages
Requires: click, instructor, langchain, langchain-openai, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [4]:
import os
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


True

## Tool

In [5]:
from exa_py import Exa
from langchain.agents import tool

class ExaSearchTool:
	@tool
	def search(query: str):
		"""Search for a webpage based on the query."""
		return ExaSearchTool._exa().search(f"{query}", use_autoprompt=True, num_results=3)

	@tool
	def find_similar(url: str):
		"""Search for webpages similar to a given URL.
		The url passed in should be a URL returned from `search`.
		"""
		return ExaSearchTool._exa().find_similar(url, num_results=3)

	@tool
	def get_contents(ids: str):
		"""Get the contents of a webpage.
		The ids must be passed in as a list, a list of ids returned from `search`.
		"""
		contents = str(ExaSearchTool._exa().get_contents(ids))
		print(contents)
		contents = contents.split("URL:")
		contents = [content[:2000] for content in contents]
		return "\n\n".join(contents)

	def tools():
		return [ExaSearchTool.search, ExaSearchTool.find_similar, ExaSearchTool.get_contents]

	def _exa():
		return Exa(api_key=os.environ["EXA_API_KEY"])

## Agents

In [6]:
from textwrap import dedent
from crewai import Agent
from langchain_openai import ChatOpenAI


## Agents
class MeetingPreparationAgents():

  def research_agent(self):
    return Agent(
        role='Research Specialist',
        goal=
        'Conduct thorough research on people and companies involved in the meeting',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As a Research Specialist, your mission is to uncover detailed information
          about the individuals and entities participating in the meeting. Your insights
          will lay the groundwork for strategic meeting preparation."""),
        llm=ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.2),
        max_rpm=240,
        verbose=True,
        memory=True)

  def industry_analysis_agent(self):
    return Agent(
        role='Tech Analyst',
        goal=
        'Analyze the current tech trends, challenges, and opportunities',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As a Tech Analyst, your analysis will identify key trends,
          challenges facing the tech, and potential opportunities that
          could be leveraged during the meeting for strategic advantage."""),
        llm=ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.4),
        max_rpm=240,
        verbose=True,
        memory=True)

  def meeting_strategy_agent(self):
    return Agent(
        role='Meeting Strategy Advisor',
        goal=
        'Develop talking points, questions, and strategic angles for the meeting',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As a Strategy Advisor, your expertise will guide the development of
          talking points, insightful questions, and strategic angles
          to ensure the meeting's objectives are achieved."""),
        llm=ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.4),
        max_rpm=10,
        verbose=True,
        memory=True)

  def summary_and_briefing_agent(self):
    return Agent(
        role='Briefing Coordinator',
        goal=
        'Compile all gathered information into a concise, informative briefing document',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As the Briefing Coordinator, your role is to consolidate the research,
          analysis, and strategic insights."""),
        llm=ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.2),
        max_rpm=10,
        verbose=True,
        memory=True)


## Tasks

In [7]:
from crewai import Task

class MeetingPreparationTasks():
  def research_task(self, agent, participants, context):
    return Task(description=dedent(f"""\
        Conduct comprehensive research on each of the individuals involved in the upcoming meeting. Gather information on recent news, achievements, professional background, and any relevant business activities. Today is April 3, 2024.

        Participants: {participants}
        Meeting Context: {context}"""),
                expected_output=dedent("""\
        A detailed report summarizing key findings about each participant, highlighting information that could be relevant for the meeting."""
                                       ),
                async_execution=True,
                agent=agent)

  def industry_analysis_task(self, agent, participants, context):
    return Task(description=dedent(f"""\
        Analyze the current tech trends, challenges, and opportunities
        relevant to the meeting's context. Consider market reports, recent
        developments, and expert opinions to provide a comprehensive
        overview of the tech landscape. Today is April 3, 2024.

        Participants: {participants}
        Meeting Context: {context}"""),
                expected_output=dedent("""\
        An insightful analysis that identifies major trends, potential
        challenges, and strategic opportunities."""),
                async_execution=True,
                agent=agent)

  def meeting_strategy_task(self, agent, context, objective):
    return Task(description=dedent(f"""\
        Develop strategic talking points, questions, and discussion angles
        for the meeting based on the research and industry analysis conducted

        Meeting Context: {context}
        Meeting Objective: {objective}"""),
                expected_output=dedent("""\
        Complete report with a list of key talking points, strategic questions
        to ask to help achieve the meetings objective during the meeting."""),
                agent=agent)

  def summary_and_briefing_task(self, agent, context, objective):
    return Task(description=dedent(f"""\
        Compile all the research findings, industry analysis, and strategic
        talking points into a concise, comprehensive briefing document for
        the meeting.
        Ensure the briefing is easy to digest and equips the meeting
        participants with all necessary information and strategies.

        Meeting Context: {context}
        Meeting Objective: {objective}"""),
                expected_output=dedent("""\
        A well-structured briefing document that includes sections for
        participant bios, industry overview, talking points, and
        strategic recommendations."""),
                agent=agent)

## Crew

In [8]:
from crewai import Crew


tasks = MeetingPreparationTasks()
agents = MeetingPreparationAgents()

participants = "Andrew Ng, Andrej Karpathy"
context = "Comptetitive analysis of 'AI Agents' technology"
objective = "Convince participants that the company should purchase Agents of Tomorrow, which is a startup that created a promising framework for orchestrating AI Agents"

In [9]:
# Create Agents
researcher_agent = agents.research_agent()
industry_analyst_agent = agents.industry_analysis_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
summary_and_briefing_agent = agents.summary_and_briefing_agent()

# Create Tasks
research = tasks.research_task(researcher_agent, participants, context)
industry_analysis = tasks.industry_analysis_task(industry_analyst_agent, participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context, objective)
summary_and_briefing = tasks.summary_and_briefing_task(summary_and_briefing_agent, context, objective)

meeting_strategy.context = [research, industry_analysis]
summary_and_briefing.context = [research, industry_analysis, meeting_strategy]


In [10]:


# Create Crew responsible for Copy
crew = Crew(
	agents=[
		researcher_agent,
		industry_analyst_agent,
		meeting_strategy_agent,
		summary_and_briefing_agent
	],
	tasks=[
		research,
		industry_analysis,
		meeting_strategy,
		summary_and_briefing
	]
)



In [24]:
result = crew.kickoff()


# Print results
print("\n\n################################################")
print("## Here is the result")
print("################################################\n")
print(result)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: To kick off the analysis, it's essential to start with identifying the current trends in AI Agents technology. Given the prominence of Andrew Ng and Andrej Karpathy in the field of AI, focusing on areas related to their expertise, such as machine learning, deep learning, and AI applications, would be beneficial. Additionally, understanding the broader market trends, challenges, and opportunities will provide a solid foundation for the analysis. The first step is to gather recent and relevant information on AI Agents technology trends.

Action: search
Action Input: {"query": "current trends in AI Agents technology 2024"}Thought: To start, I need to gather comprehensive information about Andrew Ng and Andrej Karpathy, focusing on their recent news, achievements, professional backgrounds, and any relevant business activities related to 'AI Agents' technology. I'll begin with Andrew Ng, as he i

In [25]:
from IPython.display import display, Markdown
display(Markdown(result))


**Briefing Document for Strategic Meeting on Acquisition of Agents of Tomorrow**

**Introduction**

This document serves as a comprehensive briefing for the upcoming strategic meeting focused on the potential acquisition of Agents of Tomorrow, a promising startup in the AI Agents technology space. It compiles insights from industry experts, competitive analysis, and strategic considerations to equip participants with a well-rounded understanding of the opportunity at hand.

**Participant Bios**

- **Andrew Ng:** A leading figure in AI, co-founder of Coursera, and advocate for AI's transformative potential across industries. Ng's work at Google Brain and AI Fund underscores his commitment to applying AI in innovative ways.

- **Andrej Karpathy:** Known for his contributions to computer vision and deep learning, Karpathy's tenure at Tesla and OpenAI showcases his expertise in practical AI applications and leadership in AI development.

**Industry Overview**

The AI Agents technology landscape is marked by rapid advancements and significant opportunities, alongside notable challenges. Key trends include the expansion of generative AI across sectors, AI's growing role in drug discovery, and the increasing reliability of Large Language Model (LLM) agents. However, challenges such as AI readiness and integration, and navigating a complex regulatory environment, pose hurdles to widespread adoption.

**Key Talking Points**

1. **Generative AI Expansion:** The rapid adoption of generative AI, especially in developing natural language interfaces, underscores the need to invest in cutting-edge technologies like those developed by Agents of Tomorrow.

2. **AI in Big Pharma:** AI's role in revolutionizing drug discovery processes highlights the broad applicability and potential ROI of AI technologies, supporting the case for the acquisition of Agents of Tomorrow.

3. **Reliability of LLM Agents:** Advancements in LLM-powered agents demonstrate the maturation of AI technologies, positioning Agents of Tomorrow as a valuable asset in leveraging these advancements.

**Strategic Questions**

1. **AI Readiness and Integration:** How can our investment in Agents of Tomorrow ensure we lead in AI integration and application, bridging the readiness gap?

2. **Regulatory Environment:** In light of evolving regulations, how will Agents of Tomorrow adapt its technologies to maintain innovation and competitive edge?

3. **Open-Source Innovation:** How can leveraging the open-source AI boom foster innovation and enable Agents of Tomorrow to compete against tech giants?

**Discussion Angles**

1. **Opportunity in RAG Applications:** The transition of retrieval-augmented generation applications to production opens new avenues in customer service and marketing, which Agents of Tomorrow's framework can capitalize on.

2. **Innovation and Trust:** Agents of Tomorrow's potential to innovate at the intersection of technology and trust, particularly in an era emphasizing AI ethics and transparency.

3. **Strategic Advantage through AI:** Acquiring Agents of Tomorrow can position the company at the forefront of AI technology, ensuring a competitive edge in generative AI, drug discovery, and reliable AI agents.

**Conclusion**

The acquisition of Agents of Tomorrow represents a strategic move to position the company as a leader in the AI technology space. By addressing current trends, overcoming challenges, and seizing emerging opportunities, this acquisition promises to secure a competitive advantage in the rapidly evolving AI landscape. The meeting will focus on aligning stakeholders and formulating a cohesive strategy to move forward with this acquisition.

**End of Document**